In [15]:
conda install -c conda-forge scikit-optimize

Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Natal\anaconda3

  added / updated specs:
    - scikit-optimize


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pyaml-24.4.0               |     pyhd8ed1ab_0          27 KB  conda-forge
    scikit-optimize-0.10.1     |     pyhd8ed1ab_0          81 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         108 KB

The following NEW packages will be INSTALLED:

  pyaml              conda-forge/noarch::pyaml-24.4.0-pyhd8ed1ab_0 
  scikit-optimize    conda-forge/noarch::scikit-optimize-0.10.1-pyhd8ed1ab_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2024.3.11-~ --> conda-forge::ca-certificates-2024.2.2-h



==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0




In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = XGBRegressor()

### We need to play with more than just the best performing dataframe 

In [2]:
df = pd.read_csv('df_normal_quality.csv', index_col=0)

In [3]:
y = df.SalePrice


In [4]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", drop='first')
#categorical_preprocessor2 =  OrdinalEncoder(handle_unknown="use_encoded_value", 
 #                                         unknown_value= -1) 
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("standard_scaler", numerical_preprocessor, numerical_columns),
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
    ]
)


model_pipe = Pipeline(steps=[("Preprocess", preprocessor), ("XGB", model)])

model_regr = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)

param_grid = {
    "regressor__XGB__max_depth": [2],
    "regressor__XGB__learning_rate": [.08],
    "regressor__XGB__gamma": [0],
    "regressor__XGB__reg_lambda": [.8],
    "regressor__XGB__scale_pos_weight": [1],
    "regressor__XGB__n_estimators": [630],
    "regressor__XGB__reg_alpha": [.1],
    "regressor__XGB__min_child_weight": [2]
}


search = GridSearchCV(model_regr, param_grid, n_jobs=2, cv = kf)

In [5]:
search.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=TransformedTargetRegressor(func=<ufunc 'log'>,
                                                  inverse_func=<ufunc 'exp'>,
                                                  regressor=Pipeline(steps=[('Preprocess',
                                                                             ColumnTransformer(transformers=[('standard_scaler',
                                                                                                              StandardScaler(),
                                                                                                              ['GrLivArea',
                                                                                                               'MSSubClass',
                                                                                                               'LotFrontage',
                                                                                                               'LotArea',
                                                                                                               'Alley',
                                                                                                               'LotShape',
                                                                                                               'LandContour',
                                                                                                               'Utilities',
                                                                                                               'La...
                                                                                          num_parallel_tree=None,
                                                                                          predictor=None,
                                                                                          random_state=None, ...))])),
             n_jobs=2,
             param_grid={'regressor__XGB__gamma': [0],
                         'regressor__XGB__learning_rate': [0.08],
                         'regressor__XGB__max_depth': [2],
                         'regressor__XGB__min_child_weight': [2],
                         'regressor__XGB__n_estimators': [630],
                         'regressor__XGB__reg_alpha': [0.1],
                         'regressor__XGB__reg_lambda': [0.8],
                         'regressor__XGB__scale_pos_weight': [1]})

In [6]:
print(search.best_score_)
print(search.best_params_)

0.9454823655404916
{'regressor__XGB__gamma': 0, 'regressor__XGB__learning_rate': 0.08, 'regressor__XGB__max_depth': 2, 'regressor__XGB__min_child_weight': 2, 'regressor__XGB__n_estimators': 630, 'regressor__XGB__reg_alpha': 0.1, 'regressor__XGB__reg_lambda': 0.8, 'regressor__XGB__scale_pos_weight': 1}


### above is my initial work, below will be with search grid with bayessearch

In [7]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [8]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", drop='first')
#categorical_preprocessor2 =  OrdinalEncoder(handle_unknown="use_encoded_value", 
 #                                         unknown_value= -1) 
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("standard_scaler", numerical_preprocessor, numerical_columns),
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
    ]
)


model_pipe = Pipeline(steps=[("Preprocess", preprocessor), ("XGB", model)])

model_regr = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)

param_grid = {
    "regressor__XGB__max_depth": Integer(2,5),
    "regressor__XGB__learning_rate": Real(.1, .5),
    "regressor__XGB__gamma": [0],
    "regressor__XGB__reg_lambda": Real(0, 1),
    "regressor__XGB__scale_pos_weight": [1],
    "regressor__XGB__n_estimators": [630],
    "regressor__XGB__reg_alpha": [.1],
    "regressor__XGB__min_child_weight": [2]
}


opt = BayesSearchCV(model_regr, param_grid, n_jobs=2, cv = kf)

In [9]:
opt.fit(X,y)

BayesSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
              estimator=TransformedTargetRegressor(func=<ufunc 'log'>,
                                                   inverse_func=<ufunc 'exp'>,
                                                   regressor=Pipeline(steps=[('Preprocess',
                                                                              ColumnTransformer(transformers=[('standard_scaler',
                                                                                                               StandardScaler(),
                                                                                                               ['GrLivArea',
                                                                                                                'MSSubClass',
                                                                                                                'LotFrontage',
                                                                                                                'LotArea',
                                                                                                                'Alley',
                                                                                                                'LotShape',
                                                                                                                'LandContour',
                                                                                                                'Utilities',
                                                                                                                'L...
                             'regressor__XGB__learning_rate': Real(low=0.1, high=0.5, prior='uniform', transform='normalize'),
                             'regressor__XGB__max_depth': Integer(low=2, high=5, prior='uniform', transform='normalize'),
                             'regressor__XGB__min_child_weight': [2],
                             'regressor__XGB__n_estimators': [630],
                             'regressor__XGB__reg_alpha': [0.1],
                             'regressor__XGB__reg_lambda': Real(low=0, high=1, prior='uniform', transform='normalize'),
                             'regressor__XGB__scale_pos_weight': [1]})

In [10]:
print(opt.best_score_)
print(opt.best_params_)

0.9478653411032253
OrderedDict([('regressor__XGB__gamma', 0), ('regressor__XGB__learning_rate', 0.10060991347716391), ('regressor__XGB__max_depth', 2), ('regressor__XGB__min_child_weight', 2), ('regressor__XGB__n_estimators', 630), ('regressor__XGB__reg_alpha', 0.1), ('regressor__XGB__reg_lambda', 0.049358074984923186), ('regressor__XGB__scale_pos_weight', 1)])


### feature importance below

In [51]:
from xgboost import plot_importance
#opt.best_estimator_

In [ ]:
feature_important = model.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features

In [52]:
opt.best_estimator_.

AttributeError: 'TransformedTargetRegressor' object has no attribute 'steps'